In [ ]:
import sys
sys.path.append("..")
import json
from pathlib import Path
import scraper
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
data_dir = Path("../data/")
format_dir = data_dir / "T20I"

In [ ]:


# Then for each series we can get the match metadata
for year_path in [path for path in data_dir.iterdir()]:
    with ThreadPoolExecutor() as pool:
        futures=[]
        for series_path in year_path.iterdir():
            with open(series_path / "series_meta.json", "r") as f:
                series_meta = json.load(f)

            with open(series_path / "season_meta.json", "r") as f:
                season_meta = json.load(f)
            
            series_id = series_meta["mappings"]["cricinfo"]
            
            event_urls = scraper.extract_items(scraper.json_resp("{}/{}".format(season_meta["$ref"], "events")))
            event_ids = scraper.extract_ids(event_urls)

            matches_dir = series_path / "matches" 

            for match_id, url in zip(event_ids, event_urls):
                match_dir = matches_dir / str(match_id) 
                if (match_dir / "match_meta.json").exists():
                    print("already done")
                else:
                    futures.append(pool.submit(scraper.scrape_and_save, scraper.json_resp, match_dir / "match_meta.json", True, url))
            
        [r.result() for r in as_completed(futures)]
                
                    